<img width=150 src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1a/NumPy_logo.svg/200px-NumPy_logo.svg.png"></img>

# NumPy 結構化陣列 (Structured Arrays)

除了數值資料之外，NumPy陣列也可以儲存複合式資料，也就是包含不同資料型別的元素。這就是結構化陣列 (Structured Arrays) 的功能，可以在陣列資料中指定名稱、型別，以進行後續的資料存取及處理。

範例目標:<br>
1. 分辨不同資料型別 dtype，並注意與 python 資料行別的對應
2. 實作結構化陣列 (Structured Arrays)


範例重點:<br>
1. 注意 Numpy 與 python 資料行別的對應
2. 結構化陣列可透過 dictionary 型別的資料建立 np.dtype 物件，並指定 dtype 給陣列
3. RecordArray 提供更多的屬性可以用來存取結構化陣列，但是相對的效能上較差於 Structured Array

In [1]:
import numpy as np

## 1. 資料型別 (dtype)

|資料型別|字母|Python資料型別|NumPy通用資料型別|
|---|---|---|---|
|boolean | '?'|bool|np.bool_|
|signed byte | 'b'|bytes|np.bytes_|
|unsigned byte | 'B'|bytes|np.bytes_|
|signed integer | 'i'|int|np.int_|
|unsigned integer | 'u'||np.uint|
|floating-point | 'f'|float|np.float_|
|complex-floating point | 'c'|complex|np.cfloat|
|timedelta | 'm'|datetime.timedelta|np.timedelta64|
|datetime | 'M'|datetime.datetime|np.datetime64|
|string|'S', 'a'|str|np.str_|
|Unicode string | 'U'|str|np.str_|

例如在CSV檔中有不同型別的資料要讀取。我們可以指定各個Column的資料型別。

In [ ]:
# %load structured.txt
Jay,1,2,Yes,87%
James,3,4,No,3%
Joe,5,6,Yes,55%

透過 `numpy.dtype` 物件，指定要讀入各Column的資料型別，下面的例子分別示範Python的資料型別及字母代表的型別，其中字母後的數字代表其長度，例如 f8 代表 float64 (8 bytes)，U5代表長度5以下的unicode字串。

資料型別也可以使用NumPy的資料型別，例如 `np.int32`。

In [3]:
# 使用字母代表的資料型別
dt = np.dtype('U5, i8, i8, U3')

In [4]:
a = np.genfromtxt("structured.txt", delimiter=',', dtype=dt)
a

array([('Jay', 1, 2, 'Yes'), ('James', 3, 4, 'No'), ('Joe', 5, 6, 'Yes')],
      dtype=[('f0', '<U5'), ('f1', '<i8'), ('f2', '<i8'), ('f3', '<U3')])

## 2. 結構化陣列 (Structured Arrays)

建立結構化陣列可透過dictionary型別的資料建立 `np.dtype` 物件，並指定 `dtype` 給陣列。

這邊的資料型別可以使用Python的資料型別、NumPy的資料型別、或是字母代表的型別皆可。在範例中我們混用了3種型別的表示方式。

In [7]:
dt = np.dtype({
    'names':('Name', 'num1', 'num2', 'True'), 
    'formats':((np.str_, 5), np.int32, int, 'U3')
})

In [8]:
b = np.genfromtxt("structured.txt", delimiter=',', dtype=dt)
b

array([('Jay', 1, 2, 'Yes'), ('James', 3, 4, 'No'), ('Joe', 5, 6, 'Yes')],
      dtype=[('Name', '<U5'), ('num1', '<i4'), ('num2', '<i8'), ('True', '<U3')])

建立陣列後，可以用索引的方式存取元素資料。

In [9]:
b[0]

('Jay', 1, 2, 'Yes')

也可以用Column名稱，取得Column所有元素值。

In [10]:
b['Name']

array(['Jay', 'James', 'Joe'], dtype='<U5')

或是單筆資料的欄位值。

In [11]:
b[1]['True']

'No'

也可以進行邏輯操作，取得對應的結果。

In [12]:
b[b['num2'] >= 3]['Name']

array(['James', 'Joe'], dtype='<U5')

### 建立結構化陣列

上面的範例是從檔案中讀取的資料建立結構化陣列，如果要新建立一個結構化陣列，方式跟建立陣列非常類似。

下例使用 `zeros()` 初始化陣列，並指定 `dtype`。

In [13]:
c = np.zeros(3, dtype=dt)
c

array([('', 0, 0, ''), ('', 0, 0, ''), ('', 0, 0, '')],
      dtype=[('Name', '<U5'), ('num1', '<i4'), ('num2', '<i8'), ('True', '<U3')])

In [14]:
print(c.dtype)

[('Name', '<U5'), ('num1', '<i4'), ('num2', '<i8'), ('True', '<U3')]


In [15]:
name = ['Chloe', 'Charlotte', 'Clara']
num_1 = [11, 12, 13]
num_2 = [14, 15, 16]
check = ['Y', 'Y', 'N']

In [16]:
c['Name'] = name
c['num1'] = num_1
c['num2'] = num_2
c['True'] = check

In [17]:
print(c)

[('Chloe', 11, 14, 'Y') ('Charl', 12, 15, 'Y') ('Clara', 13, 16, 'N')]


## 3. RecordArray：`numpy.recarray()`

RecordArray 與 Structured Array 非常類似，但是提供更多的屬性可以用來存取結構化陣列。不過 RecordArray 雖然方便但是在效能上會比原來的陣列差。使用方法如下：

In [18]:
c_rec = c.view(np.recarray)
c_rec

rec.array([('Chloe', 11, 14, 'Y'), ('Charl', 12, 15, 'Y'),
           ('Clara', 13, 16, 'N')],
          dtype=[('Name', '<U5'), ('num1', '<i4'), ('num2', '<i8'), ('True', '<U3')])

原先我們是透過索引或是名稱存取元素值，但是 RecordArray 可以使用屬性的方式來取得。

In [19]:
c_rec.Name

array(['Chloe', 'Charl', 'Clara'], dtype='<U5')